# 🧠 RAG with Question Transformation (Step-Back Strategy)

This notebook demonstrates how to build a smarter RAG (Retrieval-Augmented Generation) system using a method called **"Step-Back Questioning"**.

---

## 🤔 What is RAG?

RAG stands for **Retrieval-Augmented Generation**. It combines:
- **Search**: Looks up useful info from documents
- **AI Answering**: Uses a language model to write an answer based on that info

It helps answer questions even if the model doesn't "know" the answer itself.

---

## 🪜 What's Step-Back Questioning?

Sometimes user questions are too detailed or tricky for the system to retrieve the right content.  
So we apply a **Step-Back** method to turn a complex question into a **more general or easier one** first.

For example:
> Original Question: *Could the members of The Police perform lawful arrests?*  
> Step-Back Version: *What can the members of The Police do?*

This **simpler version** helps fetch better, more useful information — which can then be used to answer the original question more accurately.

---

## 📚 What This Notebook Does

### 1. Load Documents
We start by reading a PDF document and splitting it into smaller parts (called "chunks").

### 2. Build Step-Back Prompt
We design a special prompt with **examples** to teach the AI how to turn a detailed question into a general one.

### 3. Generate a Step-Back Question
We use the LLM to generate a new, broader question based on the original.

### 4. Retrieve and Generate
- Use the **step-back question** to search for relevant chunks
- Feed the found chunks along with the **original question** into the AI
- Get a smart, context-aware answer

---

## ✅ Why Use Step-Back Questioning?

- Helps find better context from documents
- Makes it easier for the AI to understand and answer
- Especially useful when the original question is unusual or very specific

---

> This is a great intro to advanced prompting techniques for Retrieval + LLMs.  
> You're teaching the AI not just *how to answer*, but also *how to ask better questions*.

## 📦 Import Required Libraries

This cell loads all the necessary tools including document loaders, prompt templates, LLMs, embeddings, and vector stores.


In [ ]:
# Import LangChain components for vector search, PDF loading, prompts, etc.
# Also load custom embedding and LLM wrappers defined in local files

from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
from llm_call import LLMCall
from embeddings import Embeddings
from operator import itemgetter

## 📄 Load and Split PDF Document

We load a sample PDF file and split it into chunks using a text splitter.  
This allows the retriever to work with smaller pieces of text and return more accurate results.


In [ ]:
# Load the PDF document
# Split it into overlapping text chunks to improve retrieval performance
pdf_file = 'sample.pdf'
chunk_size = 1000
chunk_overlap = 200

loader = PyPDFLoader(pdf_file)
documents = loader.load()

# Split the document into manageable chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)
texts = text_splitter.split_documents(documents)

/Users/chirag/miniconda3/envs/LLMs-env/lib/python3.10/site-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


In [3]:
# Show the first text chunk to inspect what the document looks like after splitting

texts[0]

Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 19.3 (Macintosh)', 'creationdate': '2024-06-18T14:09:48-07:00', 'moddate': '2024-06-18T14:10:14-07:00', 'trapped': '/False', 'source': 'sample.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='Before using iPhone, review the iPhone User Guide  at  \nsupport.apple.com/guide/iphone .\nSafety and Handling\nSee “Safety, handling, and support” in the iPhone  \nUser Guide .\nExposure to Radio Frequency\nOn iPhone, go to Settings > General > Legal &  \nRegulatory > RF Exposure. Or go to apple.com/  \nlegal/rfexposure .\nBattery and Charging\nAn iPhone battery should only be repaired by a trained \ntechnician to avoid battery damage, which could cause \noverheating, fire, or injury. Batteries should be recycled \nor disposed of separately from household waste and \naccording to local environmental laws and guidelines. For \ninformation about Apple lithium-ion batteries and battery \nservice a

In [4]:
# Check how many chunks were generated from the document

len(texts)

8

## 🧠 Define Step-Back Prompt

We build a **few-shot prompt** that helps the LLM learn how to turn specific questions into simpler, broader versions.
This "step-back" strategy boosts the effectiveness of document retrieval and generation.


In [ ]:
# Define few-shot examples to guide the LLM on how to reframe questions
# These examples show how to turn complex or niche questions into simpler/general ones
# Then, combine them into a full prompt that includes instructions + examples + actual input

from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

examples = [
    {
	"input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
	"input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]		

# We now transform these to example messages

example_prompt = ChatPromptTemplate.from_messages(
	[
		("human", "{input}"),
		("ai", "{output}"),
	]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
	example_prompt=example_prompt,
	examples=examples,
)

prompt = ChatPromptTemplate.from_messages(
	[
		("system", 
   		"""You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",),

		# Few shot examples
		few_shot_prompt,

		# New questions
		("user", "{question}"),
	]
)


# Testing the Step Back Method

In [6]:
generatye_queries_step_back = prompt | LLMCall.azure_openai() | StrOutputParser()

question = "Is there a warranty on the phone?"
queries = generatye_queries_step_back.invoke({"question": question})

In [7]:
queries

"what are the terms of the phone's warranty?"

## 💬 Define Final RAG Prompt

This prompt is used for the last step — generating a detailed answer using the retrieved chunks and the original question.

In [ ]:
# Define the prompt template for final answer generation
# This will use retrieved context + original question to guide the LLM in producing an answer

final_rag_template = """
You are a customer service agent for a apple mobile company. 
You have been given the following information about the customer question and the context.
Normal Context = {context}
Step Back Context = {step_back_context}
Customer Query: {question}

Answer: 
The answer should be based on the context provided.
Your task is to answer the customer question based on the context provided. If the question is not related to the context, please say "I don't know or Do Not Answer it just say please ask me question related to Apple Mobiles only".
Do not make up any information or provide any personal opinions or experiences.
Please answer in a friendly and professional manner.
"""

# 📝 Define the final RAG prompt template
final_rag_prompt = ChatPromptTemplate.from_template(final_rag_template)

## 🧪 Run the Full Step-Back RAG Pipeline

This is where everything comes together:
- Generate a general version of the question
- Retrieve relevant content
- Answer the original question using the found context


> 1. Generate a step-back (simpler) question from the original
> 2. Use the step-back version to retrieve better-matching documents
> 3. Feed retrieved documents + original question to LLM to generate an answer


## ☁️ Using Azure OpenAI for Embeddings & Generation

In [12]:
# 🧠 Initialize Azure OpenAI Embeddings
open_ai_embeddings = Embeddings.azure_openai()

vectorstore_openai = FAISS.from_documents(
    texts,
    open_ai_embeddings
)

retriever = vectorstore_openai.as_retriever()

# 🤖 Initialize Azure OpenAI Chat Model (LLM)
open_ai_llm = LLMCall.azure_openai()

chain = (
	{
		"context": RunnableLambda(lambda x: x["question"]) | retriever,
        "step_back_context": generatye_queries_step_back | retriever,
        "question": lambda x: x["question"],
    }
    | final_rag_prompt | open_ai_llm | StrOutputParser()

)

chain.invoke({"question": question})


'Yes, there is a warranty on the phone. Apple offers a One-Year Limited Warranty that covers defects in materials and workmanship for one year from the date of original retail purchase. However, it does not cover normal wear and tear or damage caused by accident or abuse. To obtain service under this warranty, you can call Apple or visit an Apple Store or an Apple Authorized Service Provider. Please keep in mind that available service options may depend on the country where service is requested. If you have any more questions about the warranty or need assistance, feel free to ask!'

## 🤗 Using Hugging Face for Embeddings & Generation

In [ ]:
huggingface_embeddings = Embeddings.huggingface()

vectorstore_hf = FAISS.from_documents(
    texts,
    huggingface_embeddings)

retriever_hf = vectorstore_hf.as_retriever()

huggingface_llm = LLMCall.huggingface()
chain_hf = (
    {
	"context": RunnableLambda(lambda x: x["question"]) | retriever_hf,
	"step_back_context": generatye_queries_step_back | retriever_hf,
	"question": lambda x: x["question"],
    }
    | final_rag_prompt | huggingface_llm | StrOutputParser()
)

chain_hf.invoke({"question": question})


'Yes, there is a warranty on the phone. Apple offers a One-Year Limited Warranty that covers defects in materials and workmanship for one year from the date of original retail purchase. However, it does not cover normal wear and tear or damage caused by accident or abuse. To obtain service under this warranty, you can call Apple or visit an Apple Store or an Apple Authorized Service Provider. Please note that available service options may depend on your location. For more detailed information, you can visit apple.com/legal/warranty.'

## 🦙 Using Ollama for Local LLM Inference

In [ ]:
ollama_llm = LLMCall.chat_ollama()

vectorstore_ollama = FAISS.from_documents(
    texts,
    huggingface_embeddings)

retriever_ollama = vectorstore_ollama.as_retriever()

chain_ollama = (
    {
	"context": RunnableLambda(lambda x: x["question"]) | retriever_ollama,
	"step_back_context": generatye_queries_step_back | retriever_ollama,
	"question": lambda x: x["question"],
    }
    | final_rag_prompt | ollama_llm | StrOutputParser()
)

chain_ollama.invoke({"question": question})

'Yes, there is a warranty on the phone. Apple offers a One-Year Limited Warranty that covers defects in materials and workmanship for one year from the date of original retail purchase. However, it does not cover normal wear and tear or damage caused by accident or abuse. If you need to obtain service under this warranty, you can call Apple or visit an Apple Store or an Apple Authorized Service Provider. For detailed information on obtaining service, you can visit apple.com/legal/warranty and support.apple.com.'

## ⚡ Using Groq Inference API

In [13]:
groq_llm = LLMCall.chat_groq()

vectorstore_groq = FAISS.from_documents(
    texts,
    open_ai_embeddings)

retriever_groq = vectorstore_groq.as_retriever()

chain_groq = (
    {
	"context": RunnableLambda(lambda x: x["question"]) | retriever_groq,
	"step_back_context": generatye_queries_step_back | retriever_groq,
	"question": lambda x: x["question"],
    }
    | final_rag_prompt | groq_llm | StrOutputParser()
)

chain_groq.invoke({"question": question})

"Yes, Apple provides a one-year limited warranty on its hardware products, including the iPhone, against defects in materials and workmanship. This warranty is valid for one year from the date of original retail purchase. You can find more information about the warranty and how to obtain service on Apple's website at apple.com/legal/warranty and support.apple.com."

<!-- Font Awesome CDN (Add in <head> if not already included) -->
<link
  rel="stylesheet" 
  href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.5.0/css/all.min.css"
/>

<!-- Social Footer Section -->
<div style="
  background-color:rgb(199, 195, 195);
  padding: 40px 30px;
  border-radius: 20px;
  box-shadow: 0 4px 12px rgba(0,0,0,0.08);
  font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
  font-size: 18px;
  max-width: 900px;
  margin: 60px auto 30px;
  text-align: center;
  color: #444;
">
<!-- End of Notebook Note -->
  <h2 style="margin-bottom: 10px;">📘 End of Notebook</h2>
  <p style="color: #666; font-size: 14px;">
    Thank you for exploring! Feel free to connect via the links below.
  </p>

  <!-- Social Icons -->
<div style="
  display: flex;
  gap: 25px;
  align-items: center;
  flex-wrap: wrap;
  justify-content: center;
  margin-bottom: 25px;
">
  <!-- LinkedIn -->
  <a href="https://www.linkedin.com/in/ChiragB254" target="_blank" style="text-decoration: none; color: #0077b5;">
    <i class="fab fa-linkedin fa-lg"></i> LinkedIn
  </a>

  <!-- GitHub -->
  <a href="https://github.com/ChiragB254" target="_blank" style="text-decoration: none; color: #333;">
    <i class="fab fa-github fa-lg"></i> GitHub
  </a>

  <!-- Instagram -->
  <a href="https://www.instagram.com/data.scientist_chirag" target="_blank" style="text-decoration: none; color: #E1306C;">
    <i class="fab fa-instagram fa-lg"></i> Instagram
  </a>

  <!-- Email -->
  <a href="mailto:devchirag27@gmail.com" style="text-decoration: none; color: #D44638;">
    <i class="fas fa-envelope fa-lg"></i> Email
  </a>

  <!-- X (Twitter) -->
  <a href="https://x.com/ChiragB254" target="_blank" style="text-decoration: none; color: #000;">
    <i class="fab fa-x-twitter fa-lg"></i> X.com
  </a>
  </div>

  <p style="font-size: 13px; color: black; font-style: italic; margin-top: 8px;">
    <strong>Made with ❤️ by Chirag Bansal</strong>
  </p>
</div>